Four Card Poker is owned by Shuffle Master

In [1]:
import random
import operator
import pandas as pd
import numpy as np
from itertools import combinations
from collections import Counter, OrderedDict

card_value = {"2":2, "3":3, "4":4, "5":5, "6":6, "7":7, "8":8, "9":9, "T":10, "J":11, "Q":12, "K":13, "A":14}
standard_deck = ["A♠","K♠","Q♠","J♠","T♠","9♠","8♠","7♠","6♠","5♠","4♠","3♠","2♠",
                 "A♡","K♡","Q♡","J♡","T♡","9♡","8♡","7♡","6♡","5♡","4♡","3♡","2♡",
                 "A♣","K♣","Q♣","J♣","T♣","9♣","8♣","7♣","6♣","5♣","4♣","3♣","2♣",
                 "A♢","K♢","Q♢","J♢","T♢","9♢","8♢","7♢","6♢","5♢","4♢","3♢","2♢"]

print("------------------------------")
print("       Four Card Poker       ")
print("------------------------------")

# pay table - bonus
print(" -- Ante Bonus Pay Table --")
bonus_hands = ['Four of a Kind', 'Straight Flush', 'Three of a Kind']
bonus_pays = [25, 20, 2]
bonus_pay_table = pd.DataFrame({"Hand": bonus_hands,
                                    "Payout": bonus_pays})
print(bonus_pay_table)

print("------------------------------")

# pay table - aces plus
print(" -- Aces Plus Pay Table --")
aces_plus_hands = ['Four of a Kind', 'Straight Flush', 'Three of a Kind', 'Flush', 'Straight', 'Two Pair', 'Pair of Aces']
aces_plus_pays = [50, 40, 8, 5, 4, 3, 1]
aces_plus_pay_table = pd.DataFrame({"Hand": aces_plus_hands,
                                    "Payout": aces_plus_pays})
print(aces_plus_pay_table)

print("------------------------------")

# pay table - bad beat
print(" -- Bad Beat Pay Table --")
bad_beat_hands = ['Four of a Kind', 'Straight Flush', 'Three of a Kind', 'Flush', 'Straight', 'Two Pair']
bad_beat_pays = [25000, 10000, 100, 25, 6, 4]
bad_beat_pay_table = pd.DataFrame({"Hand": bad_beat_hands,
                                    "Payout": bad_beat_pays})
print(bad_beat_pay_table)

print("------------------------------")


------------------------------
       Four Card Poker       
------------------------------
 -- Ante Bonus Pay Table --
              Hand  Payout
0   Four of a Kind      25
1   Straight Flush      20
2  Three of a Kind       2
------------------------------
 -- Aces Plus Pay Table --
              Hand  Payout
0   Four of a Kind      50
1   Straight Flush      40
2  Three of a Kind       8
3            Flush       5
4         Straight       4
5         Two Pair       3
6     Pair of Aces       1
------------------------------
 -- Bad Beat Pay Table --
              Hand  Payout
0   Four of a Kind   25000
1   Straight Flush   10000
2  Three of a Kind     100
3            Flush      25
4         Straight       6
5         Two Pair       4
------------------------------


In [159]:
# player chips
player_cash = 1000
print(f"You have ${player_cash}.")

# place initial bets
"""
# different bets: aces plus, ante, bad beat
ante_bet = abs(int(input("Please place an ante bet, in multiples of 5 or 10 dollars - minimum 5: ")))
if (ante_bet%5==0) is True:
    print("VALID")
else:
    ante_bet = abs(int(input("Try again - Please place an ante bet, in multiples of 5 or 10 dollars - minimum 5: ")))
    

aces_plus_bet = abs(int(input("Place an optional Aces Plus bet, in multiples of 5 or 10 dollars - minimum 0: ")))
if (aces_plus_bet%5==0) is True:
    print("VALID")
else:
    aces_plus_bet = abs(int(input("Try again - Place an optional Aces Plus bet, in multiples of 5 or 10 dollars - minimum 0: ")))


bad_beat_bet = abs(int(input("Place an optional Bad Beat bet, in whole dollars - minimum 0: ")))
if bad_beat_bet >= 1:
    print("VALID")
else:
    bad_beat_bet = abs(int(input("Try again - Place an optional Bad Beat bet, in whole dollars - minimum 0: ")))
"""

ante_bet = 5
aces_plus_bet = 5
bad_beat_bet = 5
player_cash = player_cash - (ante_bet + aces_plus_bet + bad_beat_bet)

print(f"You placed a ${ante_bet} ante, ${aces_plus_bet} on Aces Plus, & ${bad_beat_bet} on Bad Beat.")
print("------------------------------")
print("Cards shuffled & dealt")
# shuffle cards
random.shuffle(standard_deck)
# print(standard_deck)
print("------------------------------")


# deal cards to player & dealer
player_hand = []
dealer_hand = []

for i in range(0,5):
    player_hand.append(standard_deck[i])
    
for i in range(5,11):
    dealer_hand.append(standard_deck[i])


# test hands:
# suits ♠ ♡ ♣ ♢
# player_hand = ["4♢", "A♣", "5♢", "2♢", "3♢"] # to-do rank A234 as lowest
player_hand = ["9♣", "3♡", "2♢", "4♠", "7♢"]
dealer_hand = ["5♢", "3♡", "7♠", "T♢", "A♠", "4♡"]


# reveal player hand & dealer's up-card
print(f"Your hand: {player_hand}")
print(f"Dealer's up-card: {dealer_hand[5]}")
print("------------------------------")

# analysis
player_hand_values = [i[0] for i in player_hand]
player_hand_suits = [i[1] for i in player_hand]

player_hand_values_unique = []
player_hand_suits_unique = []

for value in player_hand_values:
    if value not in player_hand_values_unique:
        player_hand_values_unique.append(value)

player_hand_values_unique_descending = sorted(player_hand_values_unique, key= lambda x: card_value.get(x), reverse=True)
"""        
print("Player hand analysis:") 
print("------------------------------")
print("Values:")
print(f"{player_hand_values}")
print(f"(Unique, descending): {player_hand_values_unique_descending}")
print(f"{Counter(player_hand_values)}")
print("------------------------------")
print("Suits:")
print(f"{player_hand_suits}")
print(f"{Counter(player_hand_suits)}")

print("------------------------------")
# player hand RESULTS
print(f"Your cards: {player_hand}")
print("------------------------------")
"""

player_hand_best_four = []
player_hand_not_best = []
player_hand_rank = []

# four of a kind test - COMPLETE - rank=9
if max(Counter(player_hand_values).items(), key=operator.itemgetter(1))[1] == 4:
    for i in player_hand:
        if i[0] == max(Counter(player_hand_values).items(), key=operator.itemgetter(1))[0]:
            player_hand_best_four.append(i)
    print(f"You have 4 of a kind - {max(Counter(player_hand_values).items(), key=operator.itemgetter(1))[0]}'s;")
    player_hand_rank.append(9)
    player_hand_rank.append(card_value[max(Counter(player_hand_values).items(), key=operator.itemgetter(1))[0]])
    
# straight or normal flush test - NEED TO ADD META RANK - sucessfully converted AKQJT to number values
# straight flush rank=8
# normal flush rank=6
elif max(Counter(player_hand_suits).items(), key=operator.itemgetter(1))[1] >= 4:
    for i in range(len(player_hand_suits)):
        if player_hand_suits[i] == max(Counter(player_hand_suits).items(), key=operator.itemgetter(1))[0]:
            player_hand_best_four.append(player_hand[i])  
    # player_hand_best_four_values = [i[0] for i in player_hand_best_four]
    player_hand_best_four_descending = sorted([i[0] for i in player_hand_best_four], key= lambda x: card_value.get(x), reverse=True)     
    player_hand_best_four = player_hand_best_four_descending
    if len(player_hand_best_four) == 4:
        if card_value[[i[0] for i in player_hand_best_four][0]] - card_value[[i[0] for i in player_hand_best_four][3]] == 3:
            print(f"You have a straight flush - '{card_value[[i[0] for i in player_hand_best_four][0]]}' high;")
            player_hand_rank.append(8)
        elif [i[0] for i in player_hand_best_four] == ['A', '4', '3', '2']:
            print(f"You have a straight flush - '4' high;")
            player_hand_rank.append(8)
            player_hand_rank.append(4)
        else:
            print(f"You have a normal flush - '{card_value[[i[0] for i in player_hand_best_four][0]]}' high;")
            player_hand_rank.append(6)
            """player_hand_rank.append(card_value[[i[0] for i in player_hand_best_four][0]])
            player_hand_rank.append(card_value[[i[0] for i in player_hand_best_four][1]])
            player_hand_rank.append(card_value[[i[0] for i in player_hand_best_four][2]])
            player_hand_rank.append(card_value[[i[0] for i in player_hand_best_four][3]])"""
    elif len(player_hand_best_four) == 5:
        if card_value[[i[0] for i in player_hand_best_four][0]] - card_value[[i[0] for i in player_hand_best_four][3]] == 3:
            print(f"You have a straight flush - '{card_value[[i[0] for i in player_hand_best_four][0]]}' high;")
            player_hand_best_four.pop(4)
            player_hand_rank.append(8)
        elif card_value[[i[0] for i in player_hand_best_four][1]] - card_value[[i[0] for i in player_hand_best_four][4]] == 3:
            print(f"You have a straight flush - '{card_value[[i[0] for i in player_hand_best_four][1]]}' high;")
            player_hand_best_four.pop(0)
            player_hand_rank.append(8)
        elif card_value[[i[0] for i in player_hand_best_four][0]] == 14 and card_value[[i[0] for i in player_hand_best_four][0]] - card_value[[i[0] for i in player_hand_best_four][2]] == 10:
            print(f"You have a straight flush - '4' high;")
            player_hand_best_four.pop(1)
            player_hand_rank.append(8)
        else:
            print(f"You have a normal flush;")
            player_hand_rank.append(6)
            player_hand_best_four.pop(4)
    else:
        print(f"You have a normal flush;")
        player_hand_rank.append(6)
    for i in range(len(player_hand_best_four)):
        if card_value[player_hand_best_four[i]] == [14, 4, 3, 2]:
            player_hand_rank.append(4)
            player_hand_rank.append(3)
            player_hand_rank.append(2)
            player_hand_rank.append(1)
        else:
            player_hand_rank.append(card_value[player_hand_best_four[i]])
        player_hand_best_four[i] = player_hand_best_four[i] + max(Counter(player_hand_suits).items(), key=operator.itemgetter(1))[0]

# three of a kind test - INCOMPLETE - need to add fourth card to best hand, rank=7
elif max(Counter(player_hand_values).items(), key=operator.itemgetter(1))[1] == 3:
    for i in player_hand:
        if i[0] == max(Counter(player_hand_values).items(), key=operator.itemgetter(1))[0]:
            player_hand_best_four.append(i)
        else:
            player_hand_not_best.append(i)
    print(f"You have 3 of a kind - {max(Counter(player_hand_values).items(), key=operator.itemgetter(1))[0]}'s;")
    player_hand_rank.append(7)
    player_hand_rank.append(card_value[max(Counter(player_hand_values).items(), key=operator.itemgetter(1))[0]])
    if [i[0] for i in player_hand_not_best][0] >= [i[0] for i in player_hand_not_best][1]:
        player_hand_best_four.append(player_hand_not_best[0])
    else: player_hand_best_four.append(player_hand_not_best[1])
    
# normal straight test - INCOMPLETE best cards - rank=5 - else high card rank=2
elif len(player_hand_values_unique) >= 4:
    if len(player_hand_values_unique) == 5:
        if card_value[player_hand_values_unique_descending[0]] - card_value[player_hand_values_unique_descending[3]] == 3:
            print(f"You have a normal straight - '{card_value[player_hand_values_unique_descending[0]]}' high;")
            player_hand_rank.append(5)
            player_hand_rank.append(card_value[player_hand_values_unique_descending[0]])
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]-1))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]-2))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]-3))
        elif card_value[player_hand_values_unique_descending[1]] - card_value[player_hand_values_unique_descending[4]] == 3:
            print(f"You have a normal straight - '{card_value[player_hand_values_unique_descending[1]]}' high;")
            player_hand_rank.append(5)
            player_hand_rank.append(card_value[player_hand_values_unique_descending[1]])
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]-1))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]-2))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]-3))
        elif card_value[player_hand_values_unique_descending[0]] - card_value[player_hand_values_unique_descending[2]] == 10:
            print(f"You have a normal straight - '4' high;")
            player_hand_rank.append(5)
            player_hand_rank.append(4)
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]-1))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]-2))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == 14))
        else:
            print("You have high card only;")
            player_hand_rank.append(2)
            for i in range(len(player_hand_values_unique) - 1):
                player_hand_rank.append(card_value[player_hand_values_unique_descending[i]])
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[2]))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[3]))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[4]))
    elif len(player_hand_values_unique) == 4: # else one pair rank=3
        if card_value[player_hand_values_unique_descending[0]] - card_value[player_hand_values_unique_descending[3]] == 3:
            print(f"You have a normal straight - '{card_value[player_hand_values_unique_descending[0]]}' high;")
            player_hand_rank.append(5)
            player_hand_rank.append(card_value[player_hand_values_unique_descending[0]])
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]-1))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]-2))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]-3))
        elif [i[0] for i in player_hand_values_unique_descending] == ['A', '4', '3', '2']:
            print(f"You have a normal straight - '4' high;")
            player_hand_rank.append(5)
            player_hand_rank.append(4)
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]-1))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1]-2))
            player_hand_best_four.append(next(i for i in player_hand if card_value[i[0]] == 14))
        else:
            print(f"You have one pair - {max(Counter(player_hand_values).items(), key=operator.itemgetter(1))[0]}'s;")
            player_hand_rank.append(3)
            player_hand_rank.append(card_value[max(Counter(player_hand_values).items(), key=operator.itemgetter(1))[0]])

# all else is two pair rank=4
else:
    print("You have two pair;")
    player_hand_rank.append(4)
    for i in player_hand:
        if i[0] != min(Counter(player_hand_values).items(), key=operator.itemgetter(1))[0]:
            player_hand_best_four.append(i)
    if card_value[player_hand_values_unique_descending[0]] != card_value[min(Counter(player_hand_values).items(), key=operator.itemgetter(1))[0]]:
        player_hand_rank.append(card_value[player_hand_values_unique_descending[0]])
        if card_value[player_hand_values_unique_descending[1]] != card_value[min(Counter(player_hand_values).items(), key=operator.itemgetter(1))[0]]:
            player_hand_rank.append(card_value[player_hand_values_unique_descending[1]])
            if card_value[player_hand_values_unique_descending[2]] != card_value[min(Counter(player_hand_values).items(), key=operator.itemgetter(1))[0]]:
                player_hand_rank.append(card_value[player_hand_values_unique_descending[2]])
    
    
    
#player_hand_best_four_descending = sorted([i[0] for i in player_hand_best_four], key= lambda x: card_value.get(x), reverse=True)     
#player_hand_best_four = player_hand_best_four_descending    
    
print(f"Your best four cards: {player_hand_best_four}")
print(f"   Rank metadata: {player_hand_rank}")
print("------------------------------")

# print("Continue or run Simulation? ")
# play_or_fold = int(input("Would you like to Play (1, 2 , 3) times your ante or Fold (0)? "))
play_or_fold = 1

if play_or_fold == 0:
    print("You have folded your hand.")
    ante_bet_payout = 0
    aces_plus_payout = 0
elif play_or_fold >=1:
    print(f"You have Played {play_or_fold}x your ante.")
# print("------------------------------")
# print(f"Dealer's hand: {dealer_hand}")


# DEALER TESTING

print("------------------------------")
print("------------------------------")
print(f"Dealer's hand: {dealer_hand}")

# analysis
dealer_hand_values = [i[0] for i in dealer_hand]
dealer_hand_suits = [i[1] for i in dealer_hand]

dealer_hand_values_unique = []
dealer_hand_suits_unique = []

for value in dealer_hand_values:
    if value not in dealer_hand_values_unique:
        dealer_hand_values_unique.append(value)

dealer_hand_values_unique_descending = sorted(dealer_hand_values_unique, key= lambda x: card_value.get(x), reverse=True)
"""        
print("Dealer hand analysis:") 
print("------------------------------")
print("Values:")
print(f"{dealer_hand_values}")
print(f"(Unique, descending): {dealer_hand_values_unique_descending}")
print(f"{Counter(dealer_hand_values)}")
print("------------------------------")
print("Suits:")
print(f"{dealer_hand_suits}")
print(f"{Counter(dealer_hand_suits)}")
print("------------------------------")
# dealer hand RESULTS
print(f"Dealer's hand: {dealer_hand}")"""

print("------------------------------")

dealer_hand_best_four = []
dealer_hand_not_best = []
dealer_hand_rank = []


# four of a kind test - COMPLETE - rank=9
if max(Counter(dealer_hand_values).items(), key=operator.itemgetter(1))[1] == 4:
    for i in dealer_hand:
        if i[0] == max(Counter(dealer_hand_values).items(), key=operator.itemgetter(1))[0]:
            dealer_hand_best_four.append(i)
    print(f"Dealer has 4 of a kind - {max(Counter(dealer_hand_values).items(), key=operator.itemgetter(1))[0]}'s;")
    dealer_hand_rank.append(9)
    dealer_hand_rank.append(card_value[max(Counter(dealer_hand_values).items(), key=operator.itemgetter(1))[0]])


# straight flush rank=8
# normal flush rank=6
elif max(Counter(dealer_hand_suits).items(), key=operator.itemgetter(1))[1] >= 4:
    for i in range(len(dealer_hand_suits)):
        if dealer_hand_suits[i] == max(Counter(dealer_hand_suits).items(), key=operator.itemgetter(1))[0]:
            dealer_hand_best_four.append(dealer_hand[i])  
    dealer_hand_best_four_descending = sorted([i[0] for i in dealer_hand_best_four], key= lambda x: card_value.get(x), reverse=True)     
    dealer_hand_best_four = dealer_hand_best_four_descending     
    if len(dealer_hand_best_four) == 4:
        if card_value[[i[0] for i in dealer_hand_best_four][0]] - card_value[[i[0] for i in dealer_hand_best_four][3]] == 3:
            print(f"Dealer has a straight flush - '{card_value[[i[0] for i in dealer_hand_best_four][0]]}' high;")
            dealer_hand_rank.append(8)
        elif [i[0] for i in dealer_hand_best_four] == ['A', '4', '3', '2']:
            print(f"Dealer has a straight flush - '4' high;")
            dealer_hand_rank.append(8)
        elif max(Counter(dealer_hand_values).items(), key=operator.itemgetter(1))[1] == 3:
            print(f"Dealer has 3 of a kind - {max(Counter(dealer_hand_values).items(), key=operator.itemgetter(1))[0]}'s;")
            dealer_hand_rank.append(7)
            dealer_hand_rank.append(card_value[max(Counter(dealer_hand_values).items(), key=operator.itemgetter(1))[0]])
        else:
            print(f"Dealer has a normal flush;")
            dealer_hand_rank.append(6)
    elif len(dealer_hand_best_four) == 5:
        if card_value[[i[0] for i in dealer_hand_best_four][0]] - card_value[[i[0] for i in dealer_hand_best_four][3]] == 3:
            print(f"Dealer has a straight flush - '{card_value[[i[0] for i in dealer_hand_best_four][0]]}' high;")
            dealer_hand_best_four.pop(4)
            dealer_hand_rank.append(8)
        elif card_value[[i[0] for i in dealer_hand_best_four][1]] - card_value[[i[0] for i in dealer_hand_best_four][4]] == 3:
            print(f"Dealer has a straight flush - '{card_value[[i[0] for i in dealer_hand_best_four][1]]}' high;")
            dealer_hand_best_four.pop(0)
            dealer_hand_rank.append(8)
        elif card_value[[i[0] for i in dealer_hand_best_four][0]] == 14 and card_value[[i[0] for i in dealer_hand_best_four][0]] - card_value[[i[0] for i in player_hand_best_four][2]] == 10:
            print(f"Dealer has a straight flush - '4' high;")
            dealer_hand_best_four.pop(1)
            dealer_hand_rank.append(8)
        else:
            print(f"Dealer has a normal flush;")
            dealer_hand_rank.append(6)
            dealer_hand_best_four.pop(4)
    elif len(dealer_hand_best_four) == 6:
        #print("TEST SIX CARDS")
        if card_value[[i[0] for i in dealer_hand_best_four][0]] - card_value[[i[0] for i in dealer_hand_best_four][3]] == 3:
            print(f"Dealer has a straight flush - '{card_value[[i[0] for i in dealer_hand_best_four][0]]}' high;")
            dealer_hand_best_four.pop(4)
            dealer_hand_best_four.pop(4)
            dealer_hand_rank.append(8)
        elif card_value[[i[0] for i in dealer_hand_best_four][1]] - card_value[[i[0] for i in dealer_hand_best_four][4]] == 3:
            print(f"Dealer has a straight flush - '{card_value[[i[0] for i in dealer_hand_best_four][1]]}' high;")
            dealer_hand_best_four.pop(5)
            dealer_hand_best_four.pop(0)
            dealer_hand_rank.append(8)
        elif card_value[[i[0] for i in dealer_hand_best_four][2]] - card_value[[i[0] for i in dealer_hand_best_four][5]] == 3:
            print(f"Dealer has a straight flush - '{card_value[[i[0] for i in dealer_hand_best_four][2]]}' high;")
            dealer_hand_best_four.pop(0)
            dealer_hand_best_four.pop(0)
        elif card_value[[i[0] for i in dealer_hand_best_four][0]] == 14 and card_value[[i[0] for i in dealer_hand_best_four][0]] - card_value[[i[0] for i in player_hand_best_four][3]] == 10:
            print(f"Dealer has a straight flush - '4' high;")
            dealer_hand_best_four.pop(0)
            dealer_hand_best_four.pop(0)
            dealer_hand_rank.append(8)
        else:
            print(f"Dealer has a normal flush;")
            dealer_hand_best_four.pop(4)
            dealer_hand_best_four.pop(4)
            dealer_hand_rank.append(6)

    else:
        print(f"Dealer has a normal flush;")
        dealer_hand_rank.append(6)
    for i in range(len(dealer_hand_best_four)):
        if card_value[dealer_hand_best_four[i]] == [14, 4, 3, 2]:
            dealer_hand_rank.append(4)
            dealer_hand_rank.append(3)
            dealer_hand_rank.append(2)
            dealer_hand_rank.append(1)
        else:
            dealer_hand_rank.append(card_value[dealer_hand_best_four[i]])
        dealer_hand_best_four[i] = dealer_hand_best_four[i] + max(Counter(dealer_hand_suits).items(), key=operator.itemgetter(1))[0]

    if dealer_hand_rank[0] == 7:
        dealer_hand_rank.pop(2)
        dealer_hand_rank.pop(2)
        dealer_hand_rank.pop(2)
        dealer_hand_rank.pop(2)
        dealer_hand_best_four.pop(0)
        dealer_hand_best_four.pop(0)
        dealer_hand_best_four.pop(0)
        dealer_hand_best_four.pop(0)
        
# three of a kind test INCOMPLETE rank=7
elif max(Counter(dealer_hand_values).items(), key=operator.itemgetter(1))[1] == 3:
    if len(dealer_hand_values_unique) == 2:
        dealer_hand_rank.append(7)
        if card_value[dealer_hand_values_unique[0]] > card_value[dealer_hand_values_unique[1]]:
            dealer_hand_rank.append(card_value[dealer_hand_values_unique[0]]) 
            print(f"Dealer has 3 of a kind - {dealer_hand_values_unique[0]}'s;")

        else:
            dealer_hand_rank.append(card_value[dealer_hand_values_unique[1]])
            print(f"Dealer has 3 of a kind - {dealer_hand_values_unique[1]}'s;")

    else:
        for i in dealer_hand:
            if i[0] == max(Counter(dealer_hand_values).items(), key=operator.itemgetter(1))[0]:
                dealer_hand_best_four.append(i)
        print(f"Dealer has 3 of a kind - {max(Counter(dealer_hand_values).items(), key=operator.itemgetter(1))[0]}'s;")
        dealer_hand_rank.append(7)
        dealer_hand_rank.append(card_value[max(Counter(dealer_hand_values).items(), key=operator.itemgetter(1))[0]])

# normal straight test - INCOMPLETE best cards - rank=5 - else high card rank=2
elif len(dealer_hand_values_unique) >= 4:
    if len(dealer_hand_values_unique) == 4: # else two pair rank=4
        if card_value[dealer_hand_values_unique_descending[0]] - card_value[dealer_hand_values_unique_descending[3]] == 3:
            print(f"Dealer has a normal straight - '{card_value[dealer_hand_values_unique_descending[0]]}' high;")
            dealer_hand_rank.append(5)
            dealer_hand_rank.append(card_value[dealer_hand_values_unique_descending[0]])
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-1))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-2))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-3))
        elif card_value[dealer_hand_values_unique_descending[0]] - card_value[dealer_hand_values_unique_descending[2]] == 10:
            print(f"Dealer has a normal straight - '4' high;")
            dealer_hand_rank.append(5)
            dealer_hand_rank.append(4)
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-1))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-2))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == 14))
        else:
            print("Dealer has two pair;")
            dealer_hand_rank.append(4)

            
    elif len(dealer_hand_values_unique) == 5:
        if card_value[dealer_hand_values_unique_descending[0]] - card_value[dealer_hand_values_unique_descending[3]] == 3:
            print(f"Dealer has a normal straight - '{dealer_hand_rank.append(card_value[dealer_hand_values_unique_descending[0]])}' high;")
            dealer_hand_rank.append(5)
            dealer_hand_rank.append(card_value[dealer_hand_values_unique_descending[0]])
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-1))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-2))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-3))
        elif card_value[dealer_hand_values_unique_descending[1]] - card_value[dealer_hand_values_unique_descending[4]] == 3:
            print(f"Dealer has a normal straight - '{dealer_hand_rank.append(card_value[dealer_hand_values_unique_descending[1]])}' high;")
            dealer_hand_rank.append(5)
            dealer_hand_rank.append(card_value[dealer_hand_values_unique_descending[1]])
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-1))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-2))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-3))
        elif card_value[dealer_hand_values_unique_descending[0]] - card_value[dealer_hand_values_unique_descending[2]] == 10:
            print(f"Dealer has a normal straight - '4' high;")
            dealer_hand_rank.append(5)
            dealer_hand_rank.append(4)
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-1))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-2))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == 14))
        else:
            print(f"Dealer has one pair - {max(Counter(dealer_hand_values).items(), key=operator.itemgetter(1))[0]}'s;")
            dealer_hand_rank.append(3)
            dealer_hand_rank.append(card_value[max(Counter(dealer_hand_values).items(), key=operator.itemgetter(1))[0]])
    elif len(dealer_hand_values_unique) == 6:
        if card_value[dealer_hand_values_unique_descending[0]] - card_value[dealer_hand_values_unique_descending[3]] == 3:
            print(f"Dealer has a normal straight - '{card_value[dealer_hand_values_unique_descending[0]]}' high;")
            dealer_hand_rank.append(5)
            dealer_hand_rank.append(card_value[dealer_hand_values_unique_descending[0]])
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-1))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-2))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-3))
        elif card_value[dealer_hand_values_unique_descending[1]] - card_value[dealer_hand_values_unique_descending[4]] == 3:
            print(f"Dealer has a normal straight - '{card_value[dealer_hand_values_unique_descending[1]]}' high;")
            dealer_hand_rank.append(5)
            dealer_hand_rank.append(card_value[dealer_hand_values_unique_descending[1]])
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-1))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-2))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-3))
        elif card_value[dealer_hand_values_unique_descending[2]] - card_value[dealer_hand_values_unique_descending[5]] == 3:
            print(f"Dealer has a normal straight - '{card_value[dealer_hand_values_unique_descending[2]]}' high;")
            dealer_hand_rank.append(5)
            dealer_hand_rank.append(card_value[dealer_hand_values_unique_descending[2]])
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-1))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-2))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-3))
        elif card_value[dealer_hand_values_unique_descending[0]] - card_value[dealer_hand_values_unique_descending[3]] == 10:
            print(f"Dealer has a normal straight - '4' high;")
            dealer_hand_rank.append(5)
            dealer_hand_rank.append(4)
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-1))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]-2))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == 14))
        else:
            print(f"Dealer has high card only;")
            dealer_hand_rank.append(2)
            for i in range(len(dealer_hand_values_unique) - 2):
                dealer_hand_rank.append(card_value[dealer_hand_values_unique_descending[i]])
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[1]))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[2]))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[3]))
            dealer_hand_best_four.append(next(i for i in dealer_hand if card_value[i[0]] == dealer_hand_rank[4]))
elif len(dealer_hand_values_unique) == 3:
    print("Dealer has two pair;")
    dealer_hand_rank.append(4)
    dealer_hand_rank.append(card_value[dealer_hand_values_unique_descending[0]])
    dealer_hand_rank.append(card_value[dealer_hand_values_unique_descending[1]])
    
    for i in dealer_hand:
        if i[0] == dealer_hand_values_unique_descending[0] or i[0] == dealer_hand_values_unique_descending[1]:
            dealer_hand_best_four.append(i)
    
else:
    print("ERROR")
    
print(f"Dealer's best four cards: {dealer_hand_best_four}")
print(f"   Rank metadata: {dealer_hand_rank}")

print("------------------------------")
print("Result:")

if player_hand_rank == dealer_hand_rank:
    print("TIE")
elif player_hand_rank[0] > dealer_hand_rank[0]:
    print("Player wins")
elif player_hand_rank[0] < dealer_hand_rank[0]:
    print("Dealer wins")
elif player_hand_rank[0] == dealer_hand_rank[0]:
    print("TIE - testing")


You have $1000.
You placed a $5 ante, $5 on Aces Plus, & $5 on Bad Beat.
------------------------------
Cards shuffled & dealt
------------------------------
Your hand: ['9♣', '3♡', '2♢', '4♠', '7♢']
Dealer's up-card: 4♡
------------------------------
You have high card only;
Your best four cards: ['9♣', '7♢', '4♠', '3♡']
   Rank metadata: [2, 9, 7, 4, 3]
------------------------------
You have Played 1x your ante.
------------------------------
------------------------------
Dealer's cards: ['5♢', '3♡', '7♠', 'T♢', 'A♠', '4♡']
------------------------------
Dealer has high card only;
Dealer's best four cards: ['A♠', 'T♢', '7♠', '5♢']
   Rank metadata: [2, 14, 10, 7, 5]
------------------------------
Result:
TIE - testing


In [139]:
player_hand

['3♣', '3♡', '2♢', '4♠', '5♢']

In [142]:
for i in player_hand:
    print(i[0])

3
3
2
4
5


In [147]:
next(i for i in player_hand if card_value[i[0]] == player_hand_rank[1])

'5♢'

In [133]:
for i in player_hand:
    print(i[0])

3
3
2
4
5


In [96]:
player_hand_values_unique_descending

['4', '3', '2']

In [95]:
dealer_hand_values_unique_descending

['6', '5', '3']

In [112]:
card_value[dealer_hand_values_unique_descending[0]]

6

In [104]:
dealer_hand_best_four

[]

In [ ]:
if card_value[player_hand_values_unique_descending[2]] != min(Counter(player_hand_values).items(), key=operator.itemgetter(1))[0]:
                player_hand_rank.append(card_value[player_hand_values_unique_descending[2]])

In [118]:
card_value[player_hand_values_unique_descending[2]]

2

In [119]:
min(Counter(player_hand_values).items(), key=operator.itemgetter(1))[0]

'2'

In [ ]:
# TESTING dealer or player win

print(f"Rank metadata: {dealer_hand_rank}")
print(f"Player: {player_hand_rank}")
print(f"Dealer: {dealer_hand_rank}")

print("------------------------------")
print("Result:")

if player_hand_rank == dealer_hand_rank:
    print("TIE")
elif player_hand_rank[0] > dealer_hand_rank[0]:
    print("Player wins")
elif player_hand_rank[1] > dealer_hand_rank[1]:
    print("Player wins")
elif player_hand_rank[2] > dealer_hand_rank[2]:
    print("Player wins")
elif player_hand_rank[3] > dealer_hand_rank[3]:
    print("Player wins")
elif player_hand_rank[4] > dealer_hand_rank[4]:
    print("Player wins")
elif player_hand_rank[5] > dealer_hand_rank[5]:
    print("Player wins")
else:
    print("Dealer wins")

In [ ]:
# bet payouts

In [ ]:
# two pair rank=5
potential_pairs = []



In [ ]:
# two pair test